# CNN Test

### Imports

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import math
import time
import sympy
from eq_learner.DatasetCreator import DatasetCreator, TensorDataset
from eq_learner.processing import tokenization
from eq_learner.architectures.cnn import Encoder, Decoder, Seq2Seq
from sympy import sin, Symbol, log, exp 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchtext.datasets 
from torchtext.data import Field, BucketIterator

### Deterministic Results

In [3]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Cuda settings

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Generate Dataset

In [11]:
x = Symbol('x')
basis_functions = [x,sin,log,exp]
support = np.arange(0.1,3.1,0.1)
generator = DatasetCreator(basis_functions,max_linear_terms=2,max_compositions=2, random_terms=True, constants_enabled = True, constant_intervals_ext=[(-3,1),(1,3)], constant_intervals_int = [(1,3)])
res = generator.generate_set(support,50)

In [9]:
train_data = TensorDataset(x_train_n.T,y_train_p.long())

NameError: name 'x_train_n' is not defined

In [5]:
train_loader, valid_loader, test_loader, valid_idx, train_idx = dataset_loader(train_data,test_data, batch_size = 1000, valid_size = 0.30)

NameError: name 'dataset_loader' is not defined

### Encoder

### Decoder

### Seq2Seq

### Training

In [18]:

#device = torch.device('cpu')

INPUT_DIM = 1
OUTPUT_DIM = 16
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 500
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [19]:
"""def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)"""
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (linear): Linear(in_features=1, out_features=256, bias=True)
    (relu): ReLU()
    (rnn): LSTM(256, 500, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(16, 256)
    (rnn): LSTM(256, 500, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=500, out_features=16, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (relu): ReLU()
  )
)

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,052,624 trainable parameters


In [21]:
optimizer = optim.Adam(model.parameters())

In [22]:
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [23]:
"""def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch[0]
        trg = batch[1]
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        

        output = output[1:].view(-1, output_dim)
        trg = trg.T[1:].contiguous().view(-1)
        #trg = trg.T[1:].reshape(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)"""

In [24]:
"""def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[1]

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg.T[1:].contiguous().view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)"""

In [25]:
"""def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs"""

In [26]:
N_EPOCHS = 200
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_loader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [ ]:
model.eval()
with torch.no_grad():
    a = model(x_test[:1000].cuda(), y_test_p[:1000].long().cuda())

In [ ]:
n = 91

In [ ]:
ll = len((y_test[n].numpy()))
print(tokenization.get_string(np.array([12]+list(a.detach().argmax(2)[1:ll,n].cpu().numpy()))))
print(tokenization.get_string(y_test[n].numpy()))

In [ ]:
seq = y_test[n].numpy().astype('int')
print(seq)
l = len(y_test[n].numpy())
seqs = a.detach().argmax(2)[:l,n].cpu().numpy()
print(seqs)
print(seq == seqs)


In [ ]:
for j in range(len(y_test)):
    w=0
    for i in range(len(y_train)):
        try:
            if torch.sum(y_test[j] == y_train[i]) == len(y_test[j]):
                w = w+1
        except RuntimeError:
            pass
    if w == 0:
        print(j)

In [ ]:
print(y_train[2034])
print(y_test[10])